In [25]:
import pandas as pd
import os

DATA_DIR = '../data/askme-qa/'

df_10k = pd.read_csv(os.path.join(DATA_DIR, 'askme-10k.csv'))
df_full = pd.read_csv(os.path.join(DATA_DIR, 'askme-full.csv'))
df_full.head()

,_sa_instance_state_question,context,id_question,author_id,upvote,turns,text_question,paragraph_id,scope,timestamp,downvote,_sa_instance_state_paragraph,subsection_name,id_paragraph,text_paragraph,text_cleaned,is_bad,processed,subsubsection_name,page_name,section_name,section_hierarchy,word_count,within_page_order,_sa_instance_state,model,id,prompt,username,_sa_instance_state_answer,id_answer,setting,text,author_id_answer,question_id,timestamp_answer,_sa_instance_state_rating,answer_id,id_rating,timestamp_rating,author_id_rating,text_rating,value
0,<sqlalchemy.orm.state.InstanceState object at ...,In an article about 'Surface weather analysis'...,4,1,0,multi,How does the specific heat of water compare to...,176564,single-paragraph,2024-04-13 03:47:40,0,<sqlalchemy.orm.state.InstanceState object at ...,Sea and land breeze fronts,176564,Sea breeze fronts occur mainly on sunny days w...,Sea breeze fronts occur mainly on sunny days w...,False,0,NaN,Surface weather analysis,Mesoscale features,Mesoscale features > Sea and land breeze front...,103,23,<sqlalchemy.orm.state.InstanceState object at ...,mistralai/Mixtral-8x7B-Instruct-v0.1,1,Generate <int> additional short answer (DO NOT...,NaN,<sqlalchemy.orm.state.InstanceState object at ...,2,ic,"According to the article, water has a higher s...",2,4,2024-04-13 03:47:47,<sqlalchemy.orm.state.InstanceState object at ...,2,1,2024-04-13 03:47:54,3,The answer is fully accurate and detailed. It ...,5
1,<sqlalchemy.orm.state.InstanceState object at ...,In an article about 'Surface weather analysis'...,5,1,0,multi,How does the specific heat of water compare to...,176564,single-paragraph,2024-04-13 03:47:40,0,<sqlalchemy.orm.state.InstanceState object at ...,Sea and land breeze fronts,176564,Sea breeze fronts occur mainly on sunny days w...,Sea breeze fronts occur mainly on sunny days w...,False,0,NaN,Surface weather analysis,Mesoscale features,Mesoscale features > Sea and land breeze front...,103,23,<sqlalchemy.orm.state.InstanceState object at ...,mistralai/Mixtral-8x7B-Instruct-v0.1,1,Generate <int> additional short answer (DO NOT...,NaN,<sqlalchemy.orm.state.InstanceState object at ...,4,ic,"According to the article, water has a higher s...",2,5,2024-04-13 03:48:11,<sqlalchemy.orm.state.InstanceState object at ...,4,3,2024-04-13 03:48:19,3,The answer accurately summarizes the informati...,5
2,<sqlalchemy.orm.state.InstanceState object at ...,In an article about 'Surface weather analysis'...,6,1,0,multi,What happens to land temperature compared to w...,176564,single-paragraph,2024-04-13 03:47:40,0,<sqlalchemy.orm.state.InstanceState object at ...,Sea and land breeze fronts,176564,Sea breeze fronts occur mainly on sunny days w...,Sea breeze fronts occur mainly on sunny days w...,False,0,NaN,Surface weather analysis,Mesoscale features,Mesoscale features > Sea and land breeze front...,103,23,<sqlalchemy.orm.state.InstanceState object at ...,mistralai/Mixtral-8x7B-Instruct-v0.1,1,Generate <int> additional short answer (DO NOT...,NaN,<sqlalchemy.orm.state.InstanceState object at ...,7,ic,"During the day, land temperature increases sev...",2,6,2024-04-13 03:48:44,<sqlalchemy.orm.state.InstanceState object at ...,7,6,2024-04-13 03:48:50,3,The answer accurately summarizes the informati...,5
3,<sqlalchemy.orm.state.InstanceState object at ...,In an article about '2008 Maryland Terrapins f...,7,1,0,multi,Can you name the two offensive skill position ...,683013,single-paragraph,2024-04-13 03:51:53,0,<sqlalchemy.orm.state.InstanceState object at ...,Key returns,683013,Despite the loss of some of the previous year ...,Despite the loss of some of the previous year ...,False,1,NaN,2008 Maryland Terrapins football team,Before the season,Before the season > Key returns > _,146,5,<sqlalchemy.orm.state.InstanceState object at ...,mistralai/Mixtral-8x7B-Instruct-v0.1,1,Generate <int> additional short answer (DO NOT...,NaN,<sqlalchemy.orm.state.InstanceState object at ...,8,ic,"Sure, the two offensive skill po

In [31]:
import nltk
from collections import Counter
from nltk.util import ngrams
from tqdm.notebook import tqdm
import pandas as pd

nltk.download('punkt')

def get_top_ngrams(questions, n, top_k=None):
    ngram_counter = Counter()
    for question in questions:
        tokens = nltk.word_tokenize(question.lower())
        ngram_counter.update(ngrams(tokens, n))
    total = sum(ngram_counter.values())
    if not top_k:
        top_k = len(ngram_counter)
    return ngram_counter.most_common(top_k), total

def process_ngrams(questions, n_values, top_k=None):
    df_ngram = pd.DataFrame()
    for n in tqdm(n_values, desc="Processing n-grams"):
        top_ngrams, total = get_top_ngrams(questions, n, top_k)
        ngram_data = [(n, rank + 1, ' '.join(ngram), freq, freq / total) for rank, (ngram, freq) in enumerate(top_ngrams)]
        df_temp = pd.DataFrame(ngram_data, columns=['n', 'rank', 'ngram', 'frequency', 'relative_frequency'])
        df_ngram = pd.concat([df_ngram, df_temp], ignore_index=True)
    return df_ngram

questions_full = df_full['text_question'].tolist()
questions_10k = df_10k['text_question'].tolist()

if os.path.isfile(os.path.join(DATA_DIR, 'ngrams-sorted-full.csv')):
    df_ngram_full = pd.read_csv(os.path.join(DATA_DIR, 'ngrams-sorted-full.csv'))
else: 
    df_ngram_full = process_ngrams(questions_full, range(1, 9))
    df_ngram_full.to_csv(os.path.join(DATA_DIR, 'ngrams-sorted-full.csv'), index=False)

if os.path.isfile(os.path.join(DATA_DIR, 'ngrams-sorted-10k.csv')):
    df_ngram_10k = pd.read_csv(os.path.join(DATA_DIR, 'ngrams-sorted-10k.csv'))
else:
    df_ngram_10k = process_ngrams(questions_10k, range(1, 9))
    df_ngram_10k.to_csv(os.path.join(DATA_DIR, 'ngrams-sorted-10k.csv'), index=False)

[nltk_data] Downloading package punkt to
[nltk_data]     /home/yuehengzhang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [32]:
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

def calculate_ratios(df_ngram, n_values, k_values):
    ratio_sums = {k: [] for k in k_values}
    top_1_percent_ratios = []
    total_counts = []
    
    for n in n_values:
        subset = df_ngram[df_ngram['n'] == n]
        ratios = subset['relative_frequency'].tolist()
        total_count = len(ratios)
        total_counts.append(total_count)
        sorted_ratios = sorted(ratios, reverse=True)
        
        top_1_percent_count = max(1, total_count // 100)
        top_1_percent_ratios.append(sum(sorted_ratios[:top_1_percent_count]))
        
        for k in k_values:
            top_k_ratios = sorted_ratios[:k]
            ratio_sums[k].append(sum(top_k_ratios))
    
    return ratio_sums, top_1_percent_ratios, total_counts

n_values = list(range(1, 9))
k_values = [1, 2, 4, 8, 16, 32, 64, 128, 256]

full_ratios, full_top_1_percent_ratios, full_total_counts = calculate_ratios(df_ngram_full, n_values, k_values)
tenk_ratios, tenk_top_1_percent_ratios, tenk_total_counts = calculate_ratios(df_ngram_10k, n_values, k_values)

full_ratios['Top 1%'] = full_top_1_percent_ratios
full_ratios['Total Count'] = full_total_counts
tenk_ratios['Top 1%'] = tenk_top_1_percent_ratios
tenk_ratios['Total Count'] = tenk_total_counts

print("Full Dataset Ratios:")
print(pd.DataFrame(full_ratios, index=[f'{n}-gram' for n in n_values]))

print("\n10k Dataset Ratios:")
print(pd.DataFrame(tenk_ratios, index=[f'{n}-gram' for n in n_values]))

Full Dataset Ratios:
               1         2         4         8        16        32        64       128       256    Top 1%  Total Count
1-gram  0.070127  0.121722  0.192177  0.282645  0.366971  0.439122  0.495817  0.543865  0.594093  0.790285       225341
2-gram  0.012443  0.020579  0.034592  0.054085  0.076930  0.101846  0.128201  0.156221  0.188636  0.555674      2833102
3-gram  0.005985  0.009742  0.015353  0.021712  0.028955  0.036571  0.045250  0.056512  0.070583  0.349700      7746848
4-gram  0.001171  0.002155  0.003817  0.006833  0.010576  0.014652  0.019334  0.024913  0.032069  0.216721     11370385
5-gram  0.000828  0.001615  0.002929  0.004286  0.006046  0.007897  0.010061  0.013021  0.016692  0.138364     13045387
6-gram  0.000841  0.001478  0.002053  0.002509  0.003083  0.003972  0.005079  0.006533  0.008325  0.091785     13511161
7-gram  0.000106  0.000190  0.000343  0.000598  0.000971  0.001425  0.002026  0.002843  0.004050  0.066537     13283368
8-gram  0.000060  0